Bu notebook kullanıcıdan bir adet resimler içeren klasör alır. Bütün resimler öncelikle geliştirdiğimiz makine öğrenmesine sokulur. Çıktı olarak ise resimler ve resimlerin hasarlı olma ihtimallerinin bulunduğu bir pdf oluşturabilirsiniz. Oluşturulan PDF'te resimler hasar ihtimali en yüksekten en düşüğe doğru sıralanacaklardır. Ayrıca IPython Widget'ları ile notebook üzerinden resimleri tek tek inceleyebilirsiniz (Bunun için notebook'u uygun bir ortamda (lokalinizde veya Colab gibi) çalıştırmanız gerekir).

In [51]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

import os
from collections import deque
import numpy as np
import argparse
import cv2

from fpdf import FPDF

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

* imagefolder: Resşmlerinizi içeren klasör.
* output_file: Çıktı olarak oluşturulacak PDF.
* model_location: Önceden eğitilmiş modeli ağırlıkları (.h5)

In [76]:
imagefolder = 'path/to/images/folder/'
output_file = 'path/to/output.pdf'
model_location = 'path/to/model.h5'

Eğitimde kullanılan model

In [72]:
def get_model():
    # load the VGG16 network, ensuring the head FC layer sets are left
    # off
    baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

    # construct the head of the model that will be placed on top of the
    # the base model
    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(1, activation="sigmoid")(headModel)

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    model = Model(inputs=baseModel.input, outputs=headModel)

    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False

    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable
    print("[INFO] compiling model...")
    opt = SGD(lr=1e-6, momentum=0.9)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    return model

Modelin önceden eğitilmiş ağırlıklarını yüklüyoruz.

In [73]:
model = get_model()
model.load_weights(model_location)

[INFO] compiling model...


Tahmler oluşturuluyor.

In [74]:
print("[INFO] Generating Predictions...")
preds = []
filename_list = []

for filename in os.listdir(imagefolder):
    #print(filename, end=' ')
    filename_list.append(filename)
    preds_temp = []
    frame = cv2.imread(imagefolder+"//"+filename)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = frame.copy()
    frame = cv2.resize(frame, (224, 224))
    frame = frame.astype("float32")
    preds.append((output, model.predict(np.expand_dims(frame, axis=0))[0][0]))
    
preds = np.array(preds)

[INFO] Generating Predictions...


Resimleri ve tahminlerini içeren PDF oluşturuluyor. Oluşturulan PDF'te resimler hasar ihtimali en yüksekten en düşüğe doğru sıralanacaklardır.

In [77]:
print("[INFO] Generating PDF" ...")

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=20, style='B')
pdf.cell(190, 10, txt="HASAR ORANLARI", ln=1, align="C")
pdf.set_font("Arial", size=12)


for i, f_id in enumerate(preds[:,1].argsort()[::-1]):
    if(i % 2 == 0):
        if(i != 0):
            pdf.add_page()
            pdf.ln(10)
        pdf.image(imagefolder+filename_list[f_id], x=55, y=30, w=100, h=100)
        pdf.ln(110)
    else:
        pdf.image(imagefolder+filename_list[f_id], x=55, y=160, w=100, h=100)
        pdf.ln(120)
    pdf.cell(190, 10, txt="Hasar Ihtimali: %{:.2f}".format(preds[f_id][1]*100), ln=1, align='C')

pdf.output(output_file)

[INFO] Generating "hasar_oranlari.pdf" ...


''

Bu widget ile resimlerinizi ve hasar oranlarını altalta sırayla görebilirsiniz. Resimler hasar ihtimali en yüksekten en düşüğe doğru sıralanmışlardır.

In [68]:
def pred_img_widget(resim):
    i = preds[:,1].argsort()[::-1][resim]
    plt.figure(figsize=(10,10))
    plt.axis("off")
    plt.imshow(preds[i][0])
    plt.show()
    print("Hasar Ihtimali: %{:.2f}".format(preds[i][1]*100))

In [69]:
interact(pred_img_widget, resim=(0,len(preds)-1))

interactive(children=(IntSlider(value=2, description='resim', max=4), Output()), _dom_classes=('widget-interac…

<function __main__.pred_img_widget(resim)>

Bu widget ile resimlerinizi hasar ihtimalleri üzerlerine basılmış şekilde sırayla görebilirsiniz. Resimler hasar ihtimali en yüksekten en düşüğe doğru sıralanmışlardır.

In [61]:
def pred_img_txt_widget(resim):
    i = preds[:,1].argsort()[::-1][resim]
    output = preds[i][0]
    prediction = preds[i][1]
    
    text = "Hasar Ihtimali: %{:.2f}".format(prediction*100)
    x_coord = output.shape[1]//20
    y_coord = output.shape[0]//10
    font_scale = output.shape[1]/1000
    font_thickness = output.shape[1]//200
    cv2.putText(output, text, (x_coord, y_coord), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
    
    plt.figure(figsize=(10,10))
    plt.axis("off")
    plt.imshow(output)
    plt.show()

In [62]:
interact(pred_img_txt_widget, resim=(0,len(preds)-1))

interactive(children=(IntSlider(value=2, description='resim', max=4), Output()), _dom_classes=('widget-interac…

<function __main__.pred_img_txt_widget(resim)>